In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.4 MB/s eta 0:00:00:00:0100:01


In [2]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, TextStreamer, BitsAndBytesConfig, TextIteratorStreamer
import torch
import gradio as gr
import threading

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"

In [5]:
def load_model(model_name):

  quant_config = BitsAndBytesConfig(
    load_in_4bit=True, # Load model in 4-bit
    bnb_4bit_quant_type="nf4", # Use NF4 quantization
    bnb_4bit_compute_dtype=torch.bfloat16, # Compute in bfloat16
    bnb_4bit_use_double_quant=True, # Enable nested quantization
  )

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', quantization_config= quant_config)
  return tokenizer, model

In [6]:
tokenizer, model = load_model(LLAMA)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"}
]

user_input = "What is Generative AI and Agentic AI?"

max_tokens = 2000

In [11]:
def generate_stream_model(tokenizer, model, user_input, max_tokens=2000):

  global messages

  messages.append({"role": "user", "content": user_input})

  inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cuda')

  streamer = TextIteratorStreamer(
      tokenizer,
      skip_prompt=True,
      decode_kwargs={"skip_special_token":True}
  )

  thread = threading.Thread(
      target = model.generate,
      kwargs={'inputs':inputs, 'max_new_tokens':max_tokens, 'streamer':streamer}
  )
  thread.start()

  for text_chunk in streamer:
    filtered_chunk = text_chunk.replace("<|eot_id|>","")
    print(filtered_chunk, end="")

In [12]:
generate_stream_model(tokenizer, model, user_input)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>assistant<|end_header_id|>

Generative AI and Agentic AI are two related but distinct concepts in the field of artificial intelligence (AI).

**Generative AI:**

Generative AI refers to a type of AI that can generate new, original content, such as images, videos, text, or music, based on patterns, styles, or data it has learned from existing content. This is often achieved through machine learning algorithms that can learn to replicate the characteristics of existing data.

Examples of generative AI include:

* Image generation: generating realistic images from scratch using techniques like Generative Adversarial Networks (GANs) or Variational Autoencoders (VAEs)
* Text generation: generating coherent and contextually relevant text based on a given prompt or dataset
* Music generation: generating new music compositions based on patterns and styles learned from existing music

Generative AI has many applications, such as:

* Art and design: generating new art pieces, 

In [13]:
def generate_gradio_opt(user_input):

  global tokenizer, model, messages, max_tokens

  messages.append({"role": "user", "content": user_input})

  inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cuda')

  streamer = TextIteratorStreamer(
      tokenizer,
      skip_prompt=True,
      decode_kwargs={"skip_special_token":True}
  )

  thread = threading.Thread(
      target = model.generate,
      kwargs={'inputs':inputs, 'max_new_tokens':max_tokens, 'streamer':streamer}
  )
  thread.start()

  gather_reply = ""

  for text_chunk in streamer:

    filtered_chunk = text_chunk.replace("<|eot_id|>","")

    gather_reply += filtered_chunk

    yield gather_reply


  messages.append({"role": "assistant", "content": gather_reply})


In [14]:
with gr.Blocks() as demo:
  gr.Markdown("# Chat with LLAMA")
  with gr.Row():
    with gr.Column():
      user_input = gr.Textbox(label="Your Message Here", placeholder="Type Something here....")
      output_box = gr.Markdown(label="AI Response", min_height=50)
      send_button = gr.Button("Send")

  send_button.click(fn=generate_gradio_opt, inputs=user_input, outputs=output_box)


demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ad3daa0a2558922296.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ad3daa0a2558922296.gradio.live
